# Ejercicio Flask API
Para este ejercicio tendrás que desplegar un modelo de machine learning en una API para su consumo. Ya tienes un modelo entrenado,desarrollarás una API que permita consumir dicho modelo desde cualquier otra tecnología.

**Se presenta el siguiente caso de uso**

Una empresa distribuidora de ámbito nacional pretende utilizar un modelo desarrollado por el departamento de data science, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web, comunicándose con una BBDD para ingestar o reentrenar el modelo. No vale base de datos en csv. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad. (/v2/predict)
2. Un endpoint para almacenar nuevos registros en la base de datos que deberás crear previamente.(/v2/ingest_data)
3. Posibilidad de reentrenar de nuevo el modelo con los posibles nuevos registros que se recojan. (/v2/retrain)


**NOTAS**: 
1. Deberás desplegarlo desde un repositorio de github.
2. Ojo con la ruta para hacer el load de tu modelo y datos, comprueba cual es la ruta en la que está buscándolo.
3. El desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el desarrollo de una API con Flask.

**Entregable**: repositorio github.

In [55]:
import pandas as pd
import sqlite3

# Lee el archivo CSV
df = pd.read_csv('data/Advertising.csv', index_col=0)
df["newpaper"] = df["newpaper"].apply(lambda x: x.replace("s",""))

df.newpaper = df["newpaper"].astype("float")
df.info()

conn = sqlite3.connect('data/api_database.db')

# Guarda el DataFrame en la base de datos
df.to_sql('datos', conn, index=False, if_exists='replace')

# Cierra la conexión
conn.close()


<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TV        200 non-null    float64
 1   radio     200 non-null    float64
 2   newpaper  200 non-null    float64
 3   sales     200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect('api_database.db')

data = conn.execute("""
            SELECT name FROM sqlite_master WHERE type='table';

             """)
print(data.fetchall())

conn.close()

[('datos',)]


In [8]:
conn = sqlite3.connect('data/api_database.db')

data = conn.execute("""
            SELECT * FROM datos

             """)

print(data.fetchall())

[(230.1, 37.8, 69.2, 22100.0), (44.5, 39.3, 45.1, 10400.0), (17.2, 45.9, 69.3, 9300.0), (151.5, 41.3, 58.5, 18500.0), (180.8, 10.8, 58.4, 12900.0), (8.7, 48.9, 75.0, 7200.0), (57.5, 32.8, 23.5, 11800.0), (120.2, 19.6, 11.6, 13200.0), (8.6, 2.1, 1.0, 4800.0), (199.8, 2.6, 21.2, 10600.0), (66.1, 5.8, 24.2, 8600.0), (214.7, 24.0, 4.0, 17400.0), (23.8, 35.1, 65.9, 9200.0), (97.5, 7.6, 7.2, 9700.0), (204.1, 32.9, 46.0, 19000.0), (195.4, 47.7, 52.9, 22400.0), (67.8, 36.6, 114.0, 12500.0), (281.4, 39.6, 55.8, 24400.0), (69.2, 20.5, 18.3, 11300.0), (147.3, 23.9, 19.1, 14600.0), (218.4, 27.7, 53.4, 18000.0), (237.4, 5.1, 23.5, 12500.0), (13.2, 15.9, 49.6, 5600.0), (228.3, 16.9, 26.2, 15500.0), (62.3, 12.6, 18.3, 9700.0), (262.9, 3.5, 19.5, 12000.0), (142.9, 29.3, 12.6, 15000.0), (240.1, 16.7, 22.9, 15900.0), (248.8, 27.1, 22.9, 18900.0), (70.6, 16.0, 40.8, 10500.0), (292.9, 28.3, 43.2, 21400.0), (112.9, 17.4, 38.6, 11900.0), (97.2, 1.5, 30.0, 9600.0), (265.6, 20.0, 0.3, 17400.0), (95.7, 1.4, 7.

In [9]:
conn.close()

In [4]:
from flask import Flask, request, jsonify
import os
import pickle
import sqlite3
from sklearn.linear_model import LinearRegression
import pandas as pd

import requests

In [14]:
url = 'http://192.168.1.165:5000/'

requests.get(url).content

b'Bienvenido a mi API del modelo advertising'

In [5]:
def test_predict_endpoint():
    url = 'http://192.168.1.165:5000/predict'  
    data = {'data': [[100, 100, 200]]} 
    
    response = requests.get(url, json=data)
    assert response.status_code == 200
    assert 'prediction' in response.json()

In [6]:
test_predict_endpoint()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [5]:
data = {'data': [[100, 100, 200]]} 

In [6]:
def predict():
    data = request.json
    if data is None or 'data' not in data:
        return jsonify({'error': 'Datos no proporcionados o formato incorrecto'}), 400

    values = data['data']
    if not values or not isinstance(values, list) or len(values) != 3:
        return jsonify({'error': 'Formato de datos incorrecto'}), 400

    model = pickle.load(open(path_model, 'rb'))
    prediction = model.predict([[(data[0]), (data[1]), (data[2])]])
    return jsonify({'prediction': str(round(prediction[0], 2)) + 'k €'})

In [7]:
data

{'data': [[100, 100, 200]]}

In [8]:
values = data['data']

In [10]:
data

{'data': [[100, 100, 200]]}

In [9]:
values

[[100, 100, 200]]

In [19]:
path_model  = "data/advertising_model"

In [15]:
url = 'http://localhost:5000/predict'  
data = {'data': [[100, 100, 200]]} 

response = requests.get(url, json=data)


In [16]:
response

<Response [405]>

In [30]:
df = pd.DataFrame(data['data'])
df.columns = ['TV', 'radio', 'newspaper']

In [28]:
df

,tv,radio,newspaper
0,100,100,200


In [20]:
model = pickle.load(open(path_model, 'rb'))


c:\Users\Ana\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [22]:
data["data"]

[[100, 100, 200]]

In [31]:
prediction = model.predict(df)

In [32]:
prediction

array([25371.70173022])

In [ ]:
data_for_prediction = pd.DataFrame([[tv, radio, newspaper]], columns=['tv', 'radio', 'newspaper'])
prediction = model.predict(data_for_prediction)

In [ ]:
model.fit()

In [34]:
def get_data_from_database(formato):
    
    conn = sqlite3.connect("data/api_database.db")
    if formato == True:
        query = conn.execute('''SELECT tv, radio, newpaper from datos LIMIT 1 ''')
        data = query.fetchone()
    else:
        query = conn.execute('''SELECT tv, radio, newpaper, sales from datos''')
        data = query.fetchall()

    conn.close()
    return data

In [50]:
data = get_data_from_database(False)

In [51]:
df = pd.DataFrame(data)
df.columns = ['TV', 'radio', 'newpaper', 'sales']

In [52]:
df

,TV,radio,newpaper,sales
0,230.1,37.8,69.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0
...,...,...,...,...
195,38.2,3.7,13.8,7600.0
196,94.2,4.9,8.1,9700.0
197,177.0,9.3,6.4,12800.0
198,283.6,42.0,66.2,25500.0


In [53]:
X = df[['TV', 'radio', 'newpaper']]
y = df[['sales']]
model.fit(X,y)

Ridge(alpha=5500, tol=0.001)